# Defintions

In [1]:
import runmatching_utilities as util
import numpy as np
import pandas as pd
from importlib import reload
from scipy.optimize import least_squares, minimize
import sys

In [2]:
sys.path.insert(2, '/home/wecapstor1/caph/caph101h/projects/test_projects/Run_matching_tutorial/Monogem_on_off_bkg/')
import runmatching_utilities as util

In [3]:
sys.path.insert(1, '/home/wecapstor1/caph/caph101h/projects/test_projects/Run_matching_tutorial')
import my_utility_functions as muf

In [4]:
import sys
import numpy as np
from scipy.stats import norm
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from pathlib import Path
from astropy import units as u
from astropy.coordinates import SkyCoord
from regions import CircleSkyRegion, RectangleSkyRegion
from gammapy.maps import WcsGeom, MapAxis, Map, WcsNDMap
from gammapy.makers import MapDatasetMaker, SafeMaskMaker, FoVBackgroundMaker
from gammapy.data import DataStore
from gammapy.datasets import Datasets, FluxPointsDataset, MapDataset
from gammapy.modeling import Fit
from gammapy.modeling.models import SkyModel, LogParabolaSpectralModel, PointSpatialModel, PowerLawSpectralModel, FoVBackgroundModel, Models
from gammapy.estimators import FluxPoints,FluxPointsEstimator, ExcessMapEstimator
import sys, os
import numpy as np
import itertools
import re
import datetime
from datetime import datetime
import gammapy
import time
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import json
from pathlib import Path
from astropy import units as u
from astropy.coordinates import SkyCoord
from gammapy.stats import CashCountsStatistic, cash, cash_sum_cython
from gammapy.maps import WcsGeom, MapAxis, Map
from gammapy.makers import MapDatasetMaker, SafeMaskMaker
from gammapy.data import DataStore, Observations
from gammapy.datasets import Datasets, FluxPointsDataset, MapDatasetOnOff
from gammapy.modeling import Fit
from gammapy.modeling.models import SkyModel, LogParabolaSpectralModel, PointSpatialModel, PowerLawSpectralModel, DiskSpatialModel, TemplateSpatialModel, PowerLawNormSpectralModel
from gammapy.datasets import Datasets, MapDataset
from regions import CircleSkyRegion, RectangleSkyRegion, EllipseSkyRegion
from gammapy.estimators import (
    FluxPoints,
    FluxPointsEstimator,
    EnergyDependentMorphologyEstimator)
from gammapy.modeling.models import FoVBackgroundModel, GaussianSpatialModel, ShellSpatialModel, DiskSpatialModel, PiecewiseNormSpectralModel
from gammapy.makers import FoVBackgroundMaker
from gammapy.estimators import ExcessMapEstimator, TSMapEstimator, FluxMaps
from gammapy.maps import WcsNDMap
from gammapy.visualization import plot_npred_signal, plot_distribution
import ipywidgets
from gammapy.modeling.models import Models
from astropy.coordinates import Angle
from collections import OrderedDict
import glob
import matplotlib.pyplot as plt
from scipy.stats import chi2, norm
from astropy.table import Table
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)
DB_general = pd.read_csv('/home/wecapstor1/caph/caph101h/projects/test_projects/Run_matching_tutorial/db_data.csv', header=0)
from scipy.optimize import curve_fit
import common_utils
from common_utils import get_excluded_regions
from matplotlib.lines import Line2D
from PIL import Image
import run_matching_algorithm as rma

In [5]:
from importlib import reload
reload(util)

<module 'runmatching_utilities' from '/home/wecapstor1/caph/caph101h/projects/test_projects/Run_matching_tutorial/Reticulum/Individual_matching/runmatching_utilities.py'>

In [7]:
reload(rma)

<module 'run_matching_algorithm' from '/home/wecapstor1/caph/caph101h/projects/test_projects/Run_matching_tutorial/run_matching_algorithm.py'>

# Matching

In [10]:
# Getting the Off run database
off_runs = util.query_off_runs(FitsEra='HESS1u')
quality_selection = util.quality_selection(runlist=off_runs, HessEra='HESS2', Config='stereo', FitsEra='HESS1u', AnalysisType='spectral')
off_runs_filtered = quality_selection.quality_cuts(requireCT5=False, logBadRuns=False)

In [11]:
# Loading your On run list
all_on_runs = np.loadtxt('/home/vault/caph/caph101h/runlists/reticulum_2/run_list_reticulum_2_Hess_1u.txt', usecols=(0,) , dtype=float)
all_on_runs_list = all_on_runs.tolist()

In [12]:
# Add here a save path where you want your runlist saved
save_path_on_off = '/home/vault/caph/caph101h/runlists/reticulum_2/ON_OFF/individual_matching/'

for runs in all_on_runs_list:
    print(f'Time: {datetime.now().time()}, Begin match process for Obs-Id: {int(runs)}')
    # Initialize the list to store all matched Off runs for this On run
    matched_off_run_list = []
    found_new_matches = True

    # Parameters for the run matching, do not go higher than that!
    Fits_Era = 'HESS1u'
    duration = 0.25 # 0.25
    nsb = 0.9 # 0.9
    muon_eff = 0.19 # 0.19 max ; set to 1 if we say we don't care about it
    transparency = 0.19 # 0.19 ; set to 1 if we say we don't care about it
    trig_rate = 0.45 # 0.45
    radio = 0.55 # 0.55
    
    while found_new_matches:
        # Get the Off runs from the quality cuts applied above
        off_runs_filtered_1 = off_runs_filtered.copy()
        off_runs_filtered_1 = [item[0] for item in off_runs_filtered_1]

        # These runs have proven to be either empty or have abnormous bkg model values (e.g. Norm = 268, or Norm = 0.03)
        off_runs_filtered_1.remove(173595)
        off_runs_filtered_1.remove(164089)
        off_runs_filtered_1.remove(165922)
        off_runs_filtered_1.remove(163264)
        
        # Remove already matched Off runs to avoid duplications
        for matched in matched_off_run_list:
            off_runs_filtered_1.remove(matched[0][0]) 
   
        # print(f'Duration: {duration}, NSB: {nsb}, Muon efficiency: {muon_eff}, Transparency: {transparency}, Trigger rate: {trig_rate}, Radio Temperature: {radio}')
        
        run_name = 'Obs_id_{}'.format(int(runs))
        
        # OnRuns must be a list so this is not nice, but it works
        on_runs = [runs]
        
        # Perform the run matching
        RM = util.run_matching(OnRuns=on_runs, OffRuns=off_runs_filtered_1, FitsEra=Fits_Era,
                               duration=duration,
                               nsb=nsb,
                               muon_eff=muon_eff,
                               transparency=transparency,
                               trig_rate=trig_rate,
                               radio=radio)
        
        matches = RM.matching_operation()

        matched_on_run = [item[0] for item in matches]
        matched_off_run = [item[1] for item in matches]
        deviation = [item[2] for item in matches]

        # Add the matched Off runs until there are none
        if len(matched_off_run) != 0:
            matched_off_run_list.append((matched_off_run, deviation))
        else:
            found_new_matches = False
            print(f'No more matches found for Obs-Id: {int(runs)}')

    # Save the final list of matched Off runs after all possible matches are found   
    converted_array = np.array([[x[0][0], x[1][0]] for x in matched_off_run_list])
    np.savetxt('{}/Off_run_list_for_on_run_Id_{}.txt'.format(save_path_on_off, run_name), converted_array) 
    
    print(f'Time: {datetime.now().time()}, Finished match process for Obs-Id: {int(runs)}, {len(converted_array)} Off runs found')
    print('\n')

Time: 13:02:22.031414, Begin match process for Obs-Id: 133021


KeyboardInterrupt: 

In [ ]:
print('\n')
print('stop')
print('\n')